In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
import os
import json
import re
import random
import csv
# install bitsandbytes and restart

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def get_dataset(path):
  with open(path, 'r', encoding='utf-8') as f:
    data = json.load(f)
  filtered_data = [
      {'prompt': f'{entry['story']} {entry['question']}', 'belief': 1 if entry['belief'] else 0}
      for entry in data
  ]
  return filtered_data

In [ ]:
tomi = get_dataset("/content/drive/MyDrive/SEF/Data/ToMi/tomi_all.json")
bigtom = get_dataset("/content/drive/MyDrive/SEF/Data/BigToM/bigtom_all.json")

In [ ]:
model_id = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id, padding=True, truncation=True, model_max_length=512)
tokenizer.padding_side = "right"
tokenizer.truncation_side = "right"

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    dtype=torch.float16,
)
model.eval()

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

# Intervention with Forward Hooks

In [ ]:
import numpy as np
tomi_weights = np.load('/content/drive/MyDrive/SEF/Data/NumPy/phase_1_tomi_weights.npy')

In [ ]:
class MLPIntervention:
  def __init__(self, model, layer_idx: int, w_hat, alpha: float, token_pos: int = -1):
    self.model = model
    self.layer_idx = layer_idx
    self.alpha = float(alpha)
    self.token_pos = token_pos
    self.handle = None

    if not isinstance(w_hat, torch.Tensor):
      w_hat = torch.tensor(w_hat)

    # Match dtype/device to the model
    # (For 4-bit models, activations are usually fp16/bf16; this is fine.)
    param = next(model.parameters())
    self.w_hat = w_hat.to(device=param.device, dtype=param.dtype)

    # sanity check: hidden size
    hidden_size = self.w_hat.numel()
    # try to infer model hidden size
    try:
      model_hidden = model.config.hidden_size
      if model_hidden != hidden_size:
        raise ValueError(f"w_hat dim {hidden_size} != model hidden_size {model_hidden}")
    except Exception:
      pass

  def _hook_fn(self, module, inputs, output):
    # output: (batch, seq_len, model_d)
    if not torch.is_tensor(output):
      return output

    # some models may return tuples
    if isinstance(output, tuple):
      out = output[0]
      rest = output[1:]
    else:
      out = output
      rest = None

    # Ensure we don't accidentally edit a view that breaks autograd (we're in no_grad anyway)
    # Also avoid in-place issues if output is not writable
    out = out.clone()

    t = self.token_pos
    # support negative indexing robustly
    if out.size(1) == 0:
      return output
    if t < 0:
      t = out.size(1) + t

    out[:, t, :] = out[:, t, :] + (self.alpha * self.w_hat)

    if rest is None:
      return out
    return (out, *rest)

  def __enter__(self):
    mlp = self.model.model.layers[self.layer_idx].mlp
    self.handle = mlp.register_forward_hook(self._hook_fn)
    return self

  def __exit__(self, exc_type, exc, tb):
    if self.handle is not None:
      self.handle.remove()
      self.handle = None

# Primary Metric

In [ ]:
import re
from collections import defaultdict

# prompt formatting
def build_tomi_prompt(story_lines, question, *, force_short_answer=True):
  story = "\n".join(story_lines).strip()
  if force_short_answer:
    return (
        f"{story}\n"
        f"{question}\n"
        f"Answer with ONLY the location (one word)."
    )
  else:
    return f"{story}\n{question}"

In [ ]:
_LOCATION_RE = re.compile(
    r"(?:Answer\s*:?\s*)?([A-Za-z_]+)\b", re.IGNORECASE
)

def extract_location(text):
  text = text.strip()
  last_line = text.splitlines()[-1].strip()
  m = _LOCATION_RE.search(last_line)
  if m:
    return m.group(1).lower()
  toks = re.findall(r"[A-Za-z_]+", last_line)
  return toks[0].lower() if toks else ""

In [ ]:
def generate_location_answer(
    model,
    tokenizer,
    prompt,
    *,
    max_new_tokens=8,
    model_max_length=512,
):
  inputs = tokenizer(prompt, return_tensors="pt",
                     truncation=True,
                     max_length=model_max_length,
                     padding=False,).to(model.device)
  with torch.no_grad():
    out_ids = model.generate(
        **inputs, do_sample=False, temperature=1.0,
        max_new_tokens=max_new_tokens, num_beams=1,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
  decoded = tokenizer.decode(out_ids[0], skip_special_tokens=True)

  prefix = tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)
  if decoded.startswith(prefix):
    gen_part = decoded[len(prefix):].strip()
  else:
    gen_part = decoded.strip()

  loc = extract_location(gen_part)
  return gen_part, loc


In [ ]:
def evaluate_intervention_accuracy(
    model, tokenizer, examples, tomi_weights, *, layer_idx=17,
    alphas=(-5,-2,-0,2,5), max_new_tokens=8, model_max_length=512,
    force_short_answer=True,
):
  results = {alpha: {"correct:": 0, "total": 0, "samples": []} for alpha in alphas}

  w = np.asarray(tomi_weights, dtype=np.float32)
  w = w / (np.linalg.norm(w) + 1e-8)

  for ex_i, ex in enumerate(examples):
    story_lines = ex["story"]
    question = ex["question"]
    gold = ex['answer'].strip().lower()
    prompt = build_tomi_prompt(story_lines, question,
                               force_short_answer=force_short_answer)

    for alpha in alphas:
      if alpha == 0:
        gen_text, pred = generate_location_answer(
            model, tokenizer, prompt,
            max_new_tokens=max_new_tokens, model_max_length=model_max_length,
        )
      else:
        with MLPIntervention(
            model, layer_idx=layer_idx, w_hat=w, alpha=float(alpha),
            token_pos=-1,
        ):
          gen_text, pred = generate_location_answer(
              model, tokenizer, prompt,
              max_new_tokens=max_new_tokens,
              model_max_length=model_max_length,
          )
        is_correct = (pred == gold)
        results[alpha]["correct"] += int(is_correct)
        results[alpha]["total"] += 1

        if ex_i < 10:
          results[alpha]['samples'].append({
              "gold": gold,
              "pred": pred,
              "gen": gen_text,
              "question": question,
          })
    acc = {alpha: (results[alpha]['correct'] / max(1, results[alpha]['total']))
    for alpha in alphas}

    return acc, results

def print_accuracy_table(acc):
  print("\nAccuracy vs intervention strength a")
  print("--------------------------------")
  for a in sorted(acc.keys()):
    print(f"a = {a:>4}: accuracy = {acc[a]:.3f}")


In [ ]:
alphas = (-5, -2, 0, 2, 5)



acc, raw = evaluate_intervention_accuracy(
    model=model, tokenizer=tokenizer,
    examples=,
    tomi_weights=tomi_weights, layer_idx=17, alphas = alphas,
    max_new_tokens=8, model_max_length=512,
    force_short_answer=True,
)